In [1]:
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import webbrowser
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from selenium.webdriver.common.by import By

In [3]:
#=========================
# change this variable to your windows username
windows_username = "STX"
#=========================

path = 'C:\chromedriver.exe'
options = Options()
options.add_argument(f"user-data-dir=C:/Users/{windows_username}/AppData/Local/Google/Chrome/User Data/")
options.add_argument("profile-directory=selenium")
driver = webdriver.Chrome(path, options=options)

C:\Users\STX\AppData\Local\Temp\ipykernel_10832\155344705.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, options=options)


# Handle Real world website
We will be scraping and testing [activants.com](https://activants.com/)

## To avoid failed runs, we need to wait until the website is loaded to do any actions.
we will check if a certain element in a website is loaded
 - in this case, we will test if the logo is loaded.

![img](../readme_assets/activants.png)


In [6]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver.get("https://activants.com/")

wait = WebDriverWait(driver, 10)
logo = wait.until(EC.element_to_be_clickable((By.ID, 'logo')))

# Let's hoverover the Product Button

In [80]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver.get("https://activants.com/")
wait = WebDriverWait(driver, 10)
logo = wait.until(EC.element_to_be_clickable((By.ID, 'logo')))


actions = ActionChains(driver)
Products = driver.find_element(By.CLASS_NAME,'menu-item-5845')

actions.move_to_element(Products)
# actions.click()
actions.perform()

MoveTargetOutOfBoundsException: Message: move target out of bounds
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x011FACD3+2075859]
	Ordinal0 [0x0118EE61+1633889]
	Ordinal0 [0x0108B680+571008]
	Ordinal0 [0x010F38E6+997606]
	Ordinal0 [0x010D731C+881436]
	Ordinal0 [0x010EB56A+963946]
	Ordinal0 [0x010D7136+880950]
	Ordinal0 [0x010AFEFD+720637]
	Ordinal0 [0x010B0F3F+724799]
	GetHandleVerifier [0x014AEED2+2769538]
	GetHandleVerifier [0x014A0D95+2711877]
	GetHandleVerifier [0x0128A03A+521194]
	GetHandleVerifier [0x01288DA0+516432]
	Ordinal0 [0x0119682C+1665068]
	Ordinal0 [0x0119B128+1683752]
	Ordinal0 [0x0119B215+1683989]
	Ordinal0 [0x011A6484+1729668]
	BaseThreadInitThunk [0x76B26939+25]
	RtlGetFullPathName_UEx [0x777A8FD2+1218]
	RtlGetFullPathName_UEx [0x777A8F9D+1165]


# Let's inspect the document

![img](../readme_assets/inspect_activants.png)


In [75]:
Nav = driver.find_element(By.ID, 'menu-newmainmenu-1')
Nav.get_attribute('width')

In [76]:
size = Nav.size
w, h = size['width'], size['height']
w, h

(590, 54)

In [83]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver.get("https://activants.com/")
wait = WebDriverWait(driver, 10)
logo = wait.until(EC.element_to_be_clickable((By.ID, 'logo')))


actions = ActionChains(driver)
Nav = driver.find_element(By.ID, 'menu-newmainmenu-1')
Nav.get_attribute('width')
size = Nav.size
w, h = size['width'], size['height']

actions.move_to_element_with_offset(Nav, -w/5*2, 0)
# actions.click()
actions.perform()

# With math, we can solve this problem
![img](../readme_assets/select_product.png)

# Now, let's move the mouse down and click the first item in the drawer

In [85]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver.get("https://activants.com/")
wait = WebDriverWait(driver, 10)
logo = wait.until(EC.element_to_be_clickable((By.ID, 'logo')))


actions = ActionChains(driver)
Nav = driver.find_element(By.ID, 'menu-newmainmenu-1')
Nav.get_attribute('width')
size = Nav.size
w, h = size['width'], size['height']

actions.move_to_element_with_offset(Nav, -w/5*2, 0)
actions.move_by_offset(0, 30)
actions.click()
actions.perform()

# This is how it looks like
![loop](../readme_assets/whole_loop.gif)

# Extract content from a webpage

![img](../readme_assets/CDA.png)


In [112]:
first_box = driver.find_element(By.CLASS_NAME,'elementor-icon-box-content')
# check out the innerHTML
print(first_box.get_attribute('innerHTML'))



				<h3 class="elementor-icon-box-title">
					<span>
						OMNICHANNEL CAPTURE					</span>
				</h3>
									<p class="elementor-icon-box-description">
						Automates the receipt, entry, processing and output / integration of documents and electronic data in virtually any format, from any source, to any destination.					</p>
							


In [114]:
# when finding element inside an element from a element
# put a dot before //
title = first_box.find_element(By.XPATH, './/h3').get_attribute('innerText')
text = first_box.find_element(By.XPATH, './/p').get_attribute('innerText')
print(title)
print(text)

OMNICHANNEL CAPTURE
Automates the receipt, entry, processing and output / integration of documents and electronic data in virtually any format, from any source, to any destination.


# Now We want to extract the whole text block below
 - we must use find_elements to find all instances
 - then followed with a for loop to extract the information

![img](../readme_assets/activants_text.png)



In [127]:
box_list = driver.find_elements(By.CLASS_NAME,'elementor-icon-box-content')
box_list_html = [i.get_attribute('innerHTML') for i in box_list]
print(len(box_list_html))
# only the first 5 are relevant
box_list_html

8


['\n\t\t\t\t<h3 class="elementor-icon-box-title">\n\t\t\t\t\t<span>\n\t\t\t\t\t\tOMNICHANNEL CAPTURE\t\t\t\t\t</span>\n\t\t\t\t</h3>\n\t\t\t\t\t\t\t\t\t<p class="elementor-icon-box-description">\n\t\t\t\t\t\tAutomates the receipt, entry, processing and output / integration of documents and electronic data in virtually any format, from any source, to any destination.\t\t\t\t\t</p>\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t<h3 class="elementor-icon-box-title">\n\t\t\t\t\t<span>\n\t\t\t\t\t\tAUTOMATIC QUALITY ENHANCEMENT\t\t\t\t\t</span>\n\t\t\t\t</h3>\n\t\t\t\t\t\t\t\t\t<p class="elementor-icon-box-description">\n\t\t\t\t\t\tAutomatically cleans, rotates, crops and deskews documents before data is extracted and delivered into business processes.\t\t\t\t\t</p>\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t<h3 class="elementor-icon-box-title">\n\t\t\t\t\t<span>\n\t\t\t\t\t\tINTELLIGENT EXTRACTION\t\t\t\t\t</span>\n\t\t\t\t</h3>\n\t\t\t\t\t\t\t\t\t<p class="elementor-icon-box-description">\n\t\t\t\t\t\tAdvanced captur

In [128]:
title_list = []
description_list = []

for i in range(5):
    current_element = box_list[i]
    temp_title = current_element.find_element(By.XPATH, './/h3').get_attribute('innerText')
    temp_description = current_element.find_element(By.XPATH, './/p').get_attribute('innerText')
    title_list.append(temp_title)
    description_list.append(temp_description)

print(title_list), print(description_list)

['OMNICHANNEL CAPTURE', 'AUTOMATIC QUALITY ENHANCEMENT', 'INTELLIGENT EXTRACTION', 'ON-DEVICE CAPTURE', 'INTEGRATED ANALYTICS']
['Automates the receipt, entry, processing and output / integration of documents and electronic data in virtually any format, from any source, to any destination.', 'Automatically cleans, rotates, crops and deskews documents before data is extracted and delivered into business processes.', 'Advanced capture technologies that can classify, separate and extract large volumes of documents, regardless of type, content or format by leveraging machine learning technology, configurable classifiers and locators and validation rules.', "Deliver more services via your customers’ preferred channel by turning customers' smartphones and tablets into advanced document-capture devices and deliver fast, frictionless customer experience.", 'Monitor the efficiency of operations down to the granular level of user, document and index field. Where errors and bottlenecks occur, wor

(None, None)

# You are now ready to explore in the world of Selenium with Python
![img](../readme_assets/gl.gif)